In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork()

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [7]:
loss_fn = nn.CrossEntropyLoss()

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.290176  [    0/60000]
loss: 2.282043  [ 6400/60000]
loss: 2.268795  [12800/60000]
loss: 2.270091  [19200/60000]
loss: 2.239016  [25600/60000]
loss: 2.215233  [32000/60000]
loss: 2.224320  [38400/60000]
loss: 2.189327  [44800/60000]
loss: 2.183233  [51200/60000]
loss: 2.155463  [57600/60000]
Test Error: 
 Accuracy: 46.7%, Avg loss: 2.148834 

Epoch 2
-------------------------------
loss: 2.149270  [    0/60000]
loss: 2.143850  [ 6400/60000]
loss: 2.090508  [12800/60000]
loss: 2.113300  [19200/60000]
loss: 2.055496  [25600/60000]
loss: 1.993016  [32000/60000]
loss: 2.024419  [38400/60000]
loss: 1.947403  [44800/60000]
loss: 1.952072  [51200/60000]
loss: 1.881347  [57600/60000]
Test Error: 
 Accuracy: 53.1%, Avg loss: 1.878685 

Epoch 3
-------------------------------
loss: 1.901920  [    0/60000]
loss: 1.874871  [ 6400/60000]
loss: 1.768039  [12800/60000]
loss: 1.811167  [19200/60000]
loss: 1.698528  [25600/60000]
loss: 1.646938  [32000/600